In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt

In [3]:
transactions = []
with open("transactions_retail1.csv") as f:
     transactions = f.read().split("\n")   #it will split the data into different lines in reading format.
    

In [4]:
transaction_list = []
for i in transactions:
   transaction_list.append(i.split(","))

all_transaction_list = [i for item in transaction_list for i in item]
                

In [8]:
from collections import Counter # ,OrderedDict

item_frequencies = Counter(all_transaction_list)


In [9]:
item_frequencies = sorted(item_frequencies.items(), key = lambda x:x[1])

# Storing frequencies and items in separate variables 
frequencies = list(reversed([i[1] for i in item_frequencies]))
items = list(reversed([i[0] for i in item_frequencies]))


In [ ]:
# Creating Data Frame for the transactions data
transaction_series = pd.DataFrame(pd.Series(transaction_list))
transaction_series = transaction_series.iloc[:557039, :] # removing the last empty transaction

transaction_series.columns = ["items"]

# creating a dummy columns for the each item in each transactions ... Using column names as item name
X = transaction_series['items'].str.join(sep = '*').str.get_dummies(sep = '*')

X

In [ ]:

frequent_itemsets = apriori(X, min_support = 0.0075, max_len = 4, use_colnames = True)

# Most Frequent item sets based on support 
frequent_itemsets.sort_values('support', ascending = False, inplace = True)

In [ ]:
plt.bar(x = list(range(0, 11)), height = frequent_itemsets.support[0:11], color ='rgmyk');plt.xticks(list(range(0, 11)), frequent_itemsets.itemsets[0:11], rotation=20)

In [ ]:
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1)
rules.head(20)
rules.sort_values('lift', ascending = False).head(10)


In [ ]:
def to_list(i):
    return (sorted(list(i)))

ma_X = rules.antecedents.apply(to_list) + rules.consequents.apply(to_list)

ma_X = ma_X.apply(sorted)

rules_sets = list(ma_X)

unique_rules_sets = [list(m) for m in set(tuple(i) for i in rules_sets)]


In [ ]:
index_rules = []

for i in unique_rules_sets:
    index_rules.append(rules_sets.index(i))

# getting rules without any redudancy 
rules_no_redudancy = rules.iloc[index_rules, :]

# Sorting them with respect to list and getting top 10 rules 
rules_no_redudancy.sort_values('lift', ascending = False).head(10)
